<a href="https://colab.research.google.com/github/Shiddieqy/Bottani/blob/main/SIMULASI_PINN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import sys


import torch
from collections import OrderedDict

import numpy as np
import matplotlib.pyplot as plt
import scipy.io
from scipy.interpolate import griddata

from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.gridspec as gridspec
import warnings

warnings.filterwarnings('ignore')

np.random.seed(1234)




In [4]:
# CUDA support
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print(device)

cuda


In [5]:
#variables
mp=torch.tensor(1.0).float().to(device)
mt=torch.tensor(1.0).float().to(device)
gr=torch.tensor(9.8).float().to(device)

hlf=torch.tensor(0.5).float().to(device)
uni=torch.tensor(1.0).float().to(device)
zr=torch.tensor(0.0).float().to(device)
c_lyp=torch.tensor(0.0001).float().to(device)
b_lyp=torch.tensor(0.1).float().to(device)
R= torch.zeros((6,6)).float().to(device)

xd=torch.tensor(1.0).float().to(device)
ld=torch.tensor(1.0).float().to(device)

In [6]:
# the deep neural network
class DNN(torch.nn.Module):
    def __init__(self, layers):
        super(DNN, self).__init__()

        # parameters
        self.depth = len(layers) - 1

        # set up layer order dict
        self.activation = torch.nn.Tanh

        layer_list = list()
        for i in range(self.depth - 1):
            layer_list.append(
                ('layer_%d' % i, torch.nn.Linear(layers[i], layers[i+1]))
            )
            layer_list.append(('activation_%d' % i, self.activation()))

        layer_list.append(
            ('layer_%d' % (self.depth - 1), torch.nn.Linear(layers[-2], layers[-1]))
        )
        layerDict = OrderedDict(layer_list)

        # deploy layers
        self.layers = torch.nn.Sequential(layerDict)

    def forward(self, x):
        out = self.layers(x)
        return out

In [7]:
modelh = torch.load("/content/drive/MyDrive/Prof Bayu Groningen/modelRTGCPINNErrorL_E2&E5000_greaterdeg105_CL.pt")
modelh.to(device)


DNN(
  (layers): Sequential(
    (layer_0): Linear(in_features=12, out_features=20, bias=True)
    (activation_0): Tanh()
    (layer_1): Linear(in_features=20, out_features=20, bias=True)
    (activation_1): Tanh()
    (layer_2): Linear(in_features=20, out_features=20, bias=True)
    (activation_2): Tanh()
    (layer_3): Linear(in_features=20, out_features=19, bias=True)
  )
)

In [68]:
xdat = torch.tensor([0,0,1,0,0,0,0,1,1,0,0,0],dtype=torch.float32, requires_grad=True).to(device)
X = torch.tensor([1,2,1,0,0,0],dtype=torch.float32, requires_grad=True).to(device)
Ha = modelh(xdat)

In [69]:
print(Ha)

tensor([ 9.4621,  0.7816,  0.4968, -0.8280, -0.1704,  0.1826, -0.4597, -0.2600,
        -0.0973, -0.4642, -0.9170,  0.4350, -0.1108, -0.6870,  0.5383,  0.3521,
         0.1728,  0.4639,  1.4321], device='cuda:0', grad_fn=<ViewBackward0>)


In [70]:
def Hamiltonian(Xf):

      tet =Xf[0]
      x = Xf[1]
      l = Xf[2]
      ptet = Xf[3]
      px = Xf[4]
      pl = Xf[5]


      M00=mp*torch.square(l)
      M00=M00.reshape(1)
      M01=mp*torch.mul(l,torch.cos(tet))
      M01 = M01.reshape(1)
      M02=torch.zeros(tet.shape).float().to(device)
      M02 = M02.reshape(1)

      M10=mp*torch.mul(l,torch.cos(tet))
      # M10=torch.reshape(M10,(tet.shape[0],1))
      M11=torch.Tensor(mt+ mp)
      # M11=torch.reshape(M11,(tet.shape[0],1))
      M12=mp*torch.sin(tet)
      # M12=torch.reshape(M12,(tet.shape[0],1))

      M20=torch.zeros(tet.shape).float().to(device)
      # M20=torch.reshape(M20,(tet.shape[0],1))
      M21=mp*torch.sin(tet)
      # M21=torch.reshape(M21,(tet.shape[0],1))
      M22=torch.Tensor(mp)
      # M22=torch.reshape(M22,(tet.shape[0],1))
      M10 = M10.reshape(1)
      M11 = M11.reshape(1)
      M12 = M12.reshape(1)
      M20 = M20.reshape(1)
      M21 = M21.reshape(1)
      M22 = M22.reshape(1)

      # print(M00.shape)
      #Mass Matrix and Its inverse

      M0=torch.cat([M00,M01,M02])
      M1=torch.cat([M10, M11, M12])
      M2=torch.cat([M20, M21, M22])
      M=torch.cat([M0,M1,M2])
      M=torch.reshape(M,(3,3))


      Minv=torch.linalg.inv(M)
      ptet = ptet.reshape(1)
      px = px.reshape(1)
      pl = pl.reshape(1)
      p = torch.cat([ptet, px, pl],0)
      p = torch.reshape(p,(1,3))

      pT=torch.t(p)


      H=hlf*torch.matmul((torch.matmul(p, Minv)),pT)+mp*gr*l*(uni-torch.cos(tet))
      dH = torch.autograd.grad(
    H, px,
    grad_outputs=torch.ones_like(H),
    retain_graph=True,
    create_graph=True
    )[0]
      print(dH)


      return H


In [71]:

H = Hamiltonian(X)

Hd = H+ Ha + torch.square(X[1]- xd).view(H.shape) + torch.square(X[2]- ld).view(H.shape)


dH = torch.autograd.grad(
    H, X,
    grad_outputs=torch.ones_like(H),
    retain_graph=True,
    create_graph=True
)[0]
dHd = torch.autograd.grad(
    Hd, X,
    grad_outputs=torch.ones_like(Hd),
    retain_graph=True,
    create_graph=True
)[0]

tensor([0.], device='cuda:0', grad_fn=<SliceBackward0>)


In [72]:
print(dH,dHd)

tensor([8.2464, 0.0000, 4.5050, 0.0000, 0.0000, 0.0000], device='cuda:0',
       grad_fn=<AddBackward0>) tensor([156.6819,  38.0000,  85.5957,   0.0000,   0.0000,   0.0000],
       device='cuda:0', grad_fn=<AddBackward0>)
